# Train via Notebook


In [ ]:
#

In [ ]:
# Set up document and import dependencies. 
%load_ext autoreload
%autoreload 2

from pathlib import Path
from cellpose_omni import plot, models, core, io, dynamics
import omnipose, ncolor
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import time, os, sys
from tifffile import imread
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

from stardist import fill_label_holes
from cellpose_omni.models import CellposeModel

# path = r"/home/kcutler/DataDrive/omnipose_all/phase/train_sorted"
# 5I_crop data
path = '/home/kcutler/DataDrive/5I_crop/xy5/phase'
imgs_files = [str(p) for p in Path(path).glob("*.tif")]
path = '/home/kcutler/DataDrive/5I_crop/xy5/masks'
lbls_files  = [str(p) for p in Path(path).glob("*.png")]
# imgs_files = glob(os.path.join(path, "Image*.tiff"))
# lbls_files = [f.replace("Image", "segm").replace("tiff", "tif") for f in imgs_files]


basedir = '/home/kcutler/DataDrive/omnipose_all/phase/train_sorted' # replace this with wherever you put your data 
mask_filter = '_masks'
img_names = io.get_image_files(basedir,mask_filter,look_one_level_down=True)
mask_names = io.get_label_files(img_names, mask_filter)

# Obviously sorting the masks by corresponding base name is critical. The different suffixes mess this up, but sorted()
# can take a function like the one below as a key. Modifying the default suffix requires the lambda syntax (e.g. cellpose). 
def getname(path,suffix='_masks'):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

mask_names = sorted(mask_names,key=getname)
img_names = sorted(img_names,key=getname)
imgs = [skimage.io.imread(f) for f in img_names]
lbls = [ncolor.format_labels(omnipose.utils.clean_boundary(skimage.io.imread(f))) for f in mask_names]

# imgs = [skimage.io.imread(f)[np.newaxis, ...] for f in imgs_files]
# lbls = [fill_label_holes(skimage.io.imread(f)) for f in lbls_files]
print("imgs: ", [i.shape for i in imgs])
print("lbls: ", [i.shape for i in lbls])

model = CellposeModel(
    gpu=True, omni=True, nclasses=4, nchan=1, diam_mean=0,
)

In [ ]:
n = [0,1]
model.train(
    [imgs[i]  for i in n], [lbls[i] for i in n], save_every=1, n_epochs=100, batch_size=8, save_path=os.path.join(basedir, "model"),
    channels=[0,0], min_train_masks=1, rescale = 0
)

In [ ]:
k = 4
plt.imshow(plot.outline_view(imgs[k],lbls[k]))